In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict, List
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AnyMessage
import operator

In [4]:
model = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [ ]:
class AgentState(TypedDict):
    task: str # The current task human input
    plan: str # This is the plan generated by the plan agent
    draft: str # This is the draft generated by the writer agent
    critique: str # This is the critique generated by the critique agent
    content: List[str] # This is the list of documents that tavily has returned
    revision_number: int # This is the numeber of revisions made
    max_revisions: int # This is the maximum number of revisions allowed

In [5]:
import os
from typing import Dict, List, Any
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate
import requests

# Set up your API keys
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

# Initialize ChatOpenAI
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

# Define the state
class State(Dict):
    user_profile: Dict
    market_analysis: Dict
    stock_recommendations: List
    final_report: str

# User Profiler Agent
def user_profiler(state):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a financial advisor tasked with gathering user information for investment recommendations."),
        HumanMessage(content="Ask the user about their risk tolerance (conservative, moderate, aggressive), investment amount, and investment time horizon.")
    ])
    response = llm(prompt.format_messages())
    # In a real implementation, you would process user responses here
    state["user_profile"] = {
        "risk_tolerance": "moderate",
        "investment_amount": 10000,
        "time_horizon": 5
    }
    return state

# Market Analyzer Agent
def market_analyzer(state):
    def tavily_search(query):
        url = "https://api.tavily.com/search"
        params = {
            "api_key": TAVILY_API_KEY,
            "query": query,
            "search_depth": "advanced",
            "include_domains": ["finance.yahoo.com", "seekingalpha.com", "fool.com"]
        }
        response = requests.get(url, params=params)
        return response.json()

    market_data = tavily_search("current stock market trends and analysis")
    state["market_analysis"] = {
        "trends": market_data["results"][:3],  # Simplified for this example
        "overall_sentiment": "bullish"  # This would be derived from the analysis in a full implementation
    }
    return state

# Stock Screener Agent
def stock_screener(state):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a stock screening AI. Based on the user profile and market analysis, recommend 5 stocks."),
        HumanMessage(content=f"User Profile: {state['user_profile']}\nMarket Analysis: {state['market_analysis']}\nRecommend 5 stocks.")
    ])
    response = llm(prompt.format_messages())
    state["stock_recommendations"] = response.content.split("\n")  # Simplified parsing
    return state

# Report Generator Agent
def report_generator(state):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a financial report writer. Create a detailed investment report based on the given information."),
        HumanMessage(content=f"User Profile: {state['user_profile']}\nMarket Analysis: {state['market_analysis']}\nStock Recommendations: {state['stock_recommendations']}\nCreate a detailed report.")
    ])
    response = llm(prompt.format_messages())
    state["final_report"] = response.content
    return state

# Define the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("user_profiler", user_profiler)
workflow.add_node("market_analyzer", market_analyzer)
workflow.add_node("stock_screener", stock_screener)
workflow.add_node("report_generator", report_generator)

# Add edges
workflow.add_edge("user_profiler", "market_analyzer")
workflow.add_edge("market_analyzer", "stock_screener")
workflow.add_edge("stock_screener", "report_generator")
workflow.add_edge("report_generator", END)

# Set the entry point
workflow.set_entry_point("user_profiler")

# Compile the graph
app = workflow.compile()

# Run the graph
inputs = {}  # You can add initial inputs here if needed
result = app.invoke(inputs)

print(result["final_report"])

InvalidUpdateError: Must write to at least one of ['user_profile', 'market_analysis', 'stock_recommendations', 'final_report']